1.1:

I think that if it is possible to create an artificial intelligence capable of thinking humanly, it would definitely need to have the capability of introspection. I think this is quite clear by definition; humans are able to introspect, therefore artificial intelligence to mimic humans should also have introspection. Of course, this question hinges on the idea that AI will eventually be able to fully (or nearly) copy human thought.

The more interesting question, then, is whether introspection has any bearing on current state of AI. It is certainly concievable that an AI equipped with introspection may be able to outclass another by using introspection to understand when it's making a mistake such as a false assumption. The bigger issue, though, is that nueral network based AI tends to be highly performance heavy, especially during the training process. Because of this, it is common practice to reduce the input data as much as possible while still retaining the necessary information. By this logic, adding the capacity for introspection would likely increase the complexity of the AI, making it slower to train and less likely to find the best solutions.

Another question here is whether we can even choose whether introspection is possible (at least in the realm of neural networks). A NN with enough neurons could build up a solution generating portion, and devote the rest to monitor the process, essentially being introspective to the thought process. Of course, the introspection would have to be helpful in order for it to develop through standard training methods, which would kind of answer the question itself.

Overall, I do think that the idea of introspection is important to consider when thinking about AI, but whether or not it is a good practice or model has yet to be seen as far as I'm aware.

1.2:

The state is a list describing the order in which the cities are visited.
The map is a list of cities, which are represented as a list of distances to the other cities
Actions involve swapping the order of two cities

In [1]:


from search import Problem, hill_climbing, simulated_annealing, exp_schedule
import itertools
import math


class TSP(Problem):
    """An implementation of Travelling Salesperson Problem

    State representation:
        [n1, n2, n3...] gives the order in which the cities are visited
    Move representation:
        [x, y]: Swap the order of visit x with visit y
    """

    def __init__(self):
        """The distance from node x to node y is map[x][y] or map[y][x]"""
        self.map = [(0, 11, 29, 15, 6), (11, 0, 37, 42, 12), (29, 37, 0, 54, 2), (15, 42, 54, 0, 6), (6, 12, 2, 6, 0)]
        self.initial = [0, 1, 2, 3, 4]

    def actions(self, state):
        """Actions will swap the order of two moves
        """
        actions = []
        for i in range(len(self.initial)):
            for j in range(len(self.initial)):
                if i != j:
                    actions.append([i, j])
        return actions

    def result(self, state, move):
        """Makes the given swap on a copy of the given state."""
        new_state = list(state)
        i = new_state[move[0]]
        new_state[move[0]] = new_state[move[1]]
        new_state[move[1]] = i;
        return new_state

    def value(self, state):
        """This method computes a value of given state based on the negative distance
        """
        value = 0
        for i in range(len(state)):
            value -= self.map[state[i]][state[(i + 1) % len(state)]]

        return value

In [3]:
p = TSP()

# Solve the problem using hill-climbing.
hill_solution = hill_climbing(p)
print('Hill-climbing solution       x: ' + str(hill_solution)
      + '\tvalue: ' + str(p.value(hill_solution))
      )

# Solve the problem using simulated annealing.
annealing_solution = simulated_annealing(
    p,
    exp_schedule(k=20, lam=0.005, limit=1000)
)
print('Simulated annealing solution x: ' + str(annealing_solution)
      + '\tvalue: ' + str(p.value(annealing_solution))
      )


Hill-climbing solution       x: [2, 1, 0, 3, 4]	value: -71
Simulated annealing solution x: [1, 0, 3, 4, 2]	value: -71


1.3
I chose this implementation because it was the only one that I could get to work.

In [10]:
from csp import min_conflicts, backtracking_search, AC3, CSP, forward_checking
from search import depth_first_graph_search
from collections import defaultdict

def Schedule():
    """Return an instance of the Schedule problem."""
    Profs = 'Plantinga Norman Schuurman Adams'.split()
    Courses = '108 112 212 214 336 384 395'.split()
    Times = '9mwf 1030mwf 1130mwf 1030tth 130tth'.split()
    Rooms = 'nh253 sb384'.split()
    variables = list(Courses)
    domainsList = []
    for x in Profs:
        for y in Times:
            for z in Rooms:
                # Append the domains as a comma-separated string
                domainsList.append(x + "," + y + "," + z)
                
    domains = {}
    for var in variables:
        domains[var] = domainsList

    # Generate list of neighbors
    neighbors = defaultdict(list)
    for type in [Courses, domainsList]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    # Prints out debug information as the program runs (if class_constraint is replaced)
    def dbg(A, a, B, b):
        print(A, a, B, b)
        s = class_constraint(A, a, B, b)
        print(s)
        return s

    # Checks constraints
    def class_constraint(A, a, B, b):
        al = a.split(',')
        bl = b.split(',')

        # if Prof the same, ensure time is different
        if al[0] == bl[0] and al[1] == bl[1]:
            return False
        # if Time the same, ensure room and prof are different
        if al[1] == bl[1]:
            if al[2] == bl[2] or al[0] == bl[0]:
                return False

        # if Room the same, ensure time is different
        if al[2] == bl[2] and al[1] == bl[1]:
            return False

        # Ensure all professors teach the correct classes
        if al[0] == 'Plantinga' and A not in ('212', '395'):
            return False
        if al[0] == 'Norman' and A not in ('336', '108'):
            return False
        if al[0] == 'Schuurman' and A not in ('384', '108'):
            return False
        if al[0] == 'Adams' and A not in ('112', '214'):
            return False
        if bl[0] == 'Plantinga' and B not in ('212', '395'):
            return False
        if bl[0] == 'Norman' and B not in ('336', '108'):
            return False
        if bl[0] == 'Schuurman' and B not in ('384', '108'):
            return False
        if bl[0] == 'Adams' and B not in ('112', '214'):
            return False
        
        # If no constraints failed, return true
        return True
    
    return CSP(variables, domains, neighbors, class_constraint)

In [11]:
problem = Schedule()

solution = backtracking_search(problem, inference=forward_checking)
#solution = min_conflicts(problem)

print(solution)


{'212': 'Plantinga,1030mwf,nh253', '384': 'Schuurman,1130mwf,sb384', '214': 'Adams,1030mwf,sb384', '336': 'Norman,1130mwf,nh253', '395': 'Plantinga,1030tth,nh253', '112': 'Adams,9mwf,sb384', '108': 'Norman,9mwf,nh253'}
